# Adding NGOs we are researching to web archive

Web archive allows to make snapshots of a web page and save it in its current state. We want to save all pages of each NGO website as a snapshot (also called memento in official docs https://archive.readme.io/docs/creating-a-snapshot). The snapshots are stored indefinetely, however, if the official owner of the website formally asks, a snapshot can be deleted from the web archive, although I suspect that not many people generally know about this.

First step we load the data

### Data

In [1]:
import pandas as pd
import requests, json, os
from json.decoder import JSONDecodeError
import json

In [2]:
data_url = "https://raw.githubusercontent.com/Teplitsa/CSRLab/main/data/2022_lab_index_report.csv"
index_report_data = pd.read_csv(data_url, encoding='utf-8')
index_report_data.tail()

,ogrn,website,bin_robots,bin_sitemap,bin_fb,bin_vk,bin_ig,bin_ok,bin_youtube,bin_tiktok,bin_donation,wcag_score,mean_page_speed,bin_title,bin_headings,bin_mob_friendly,bin_ssl,bin_socnet
9540,1217700385539,ngolikeyou.ru,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.884615,0.970000,1.0,0.0,1.0,NaN,0.666667
9541,1214600009524,kcmol.ru,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.812500,0.573333,1.0,1.0,1.0,0.0,0.333333
9542,1217700396330,gorneks.ru,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.934783,0.780000,1.0,0.0,NaN,1.0,0.000000
9543,1213300006920,ano-nachalo.ru,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.896552,0.996667,1.0,1.0,NaN,0.0,0.000000
9544,1212300052249,blagfond-zzh.ru,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.000000,0.413333,1.0,1.0,NaN,1.0,0.000000


In [3]:
len(index_report_data)

9545

In [4]:
data_url = "https://raw.githubusercontent.com/Teplitsa/CSRLab/main/data/2022_lab_selected_ngos_data.csv"
selected_ngos_data = pd.read_csv(data_url, encoding='utf-8')
selected_ngos_data.head()

,ogrn,website,regionName,minjustForm,shortName,regDate,mainOkved,opfCombined,okvedCombined,vedSimple,websiteCreationDate
0,1026800000666,profobr68.ru,Тамбовская область,Профессиональный союз,ТАМБОВСКАЯ ОБЛАСТНАЯ ОРГАНИЗАЦИЯ ОБЩЕРОССИЙСКО...,1938-11-10,Деятельность профессиональных союзов,Профсоюзные организации,Деятельность профессиональных союзов,Профсоюзы,2016-01-15 19:31:34
1,1022900002861,проф-севмаш.рф,Архангельская область,Профессиональный союз,"МОО - ППО АО ""ПО ""СЕВМАШ"" СУДПРОФ",1942-03-15,Деятельность профессиональных союзов,Профсоюзные организации,Деятельность профессиональных союзов,Профсоюзы,2018-08-27 11:35:06
2,1025200008140,vdpor52.ru,Нижегородская область,Общественная организация,"НИЖЕГОРОДСКОЕ ОБЛАСТНОЕ ОТДЕЛЕНИЕ ВДПО, ВДПО Н...",1958-06-20,"Деятельность прочих общественных организаций, ...",Общественные организации,"Деятельность прочих общественных организаций, ...",Общественные организации,2016-09-28 12:05:06
3,1037739471220,orthonet.ru,Москва,Общественная организация,СОЮЗ ПИСАТЕЛЕЙ РОССИИ,1958-12-07,Деятельность профессиональных членских организ...,Общественные организации,Деятельность профессиональных членских организ...,Профсоюзы,2005-11-24 21:00:00
4,1085200004450,vdpo-sarov.ru,Нижегородская область,Общественная организация,"САРОВСКОЕ ""ВДПО""",1962-03-30,"Деятельность прочих общественных организаций, ...",Общественные организации,"Деятельность прочих общественных организаций, ...",Общественные организации,2021-04-15 8:10:11


In [5]:
len(selected_ngos_data)

9545

It the same dataset just with different columns. It's kind of strange that it is split into two dataframes, we are going to merge it in one based on two columns -- *website* and *ogrn*.

In [6]:
concat_ngos = pd.merge(
    index_report_data,
    selected_ngos_data,
    how="outer",
    on=["ogrn", "website"],
    sort=False,
    left_index=False,
    right_index=False,
)

In [7]:
concat_ngos.head()

,ogrn,website,bin_robots,bin_sitemap,bin_fb,bin_vk,bin_ig,bin_ok,bin_youtube,bin_tiktok,...,bin_socnet,regionName,minjustForm,shortName,regDate,mainOkved,opfCombined,okvedCombined,vedSimple,websiteCreationDate
0,1026800000666,profobr68.ru,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,Тамбовская область,Профессиональный союз,ТАМБОВСКАЯ ОБЛАСТНАЯ ОРГАНИЗАЦИЯ ОБЩЕРОССИЙСКО...,1938-11-10,Деятельность профессиональных союзов,Профсоюзные организации,Деятельность профессиональных союзов,Профсоюзы,2016-01-15 19:31:34
1,1022900002861,проф-севмаш.рф,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.166667,Архангельская область,Профессиональный союз,"МОО - ППО АО ""ПО ""СЕВМАШ"" СУДПРОФ",1942-03-15,Деятельность профессиональных союзов,Профсоюзные организации,Деятельность профессиональных союзов,Профсоюзы,2018-08-27 11:35:06
2,1025200008140,vdpor52.ru,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,Нижегородская область,Общественная организация,"НИЖЕГОРОДСКОЕ ОБЛАСТНОЕ ОТДЕЛЕНИЕ ВДПО, ВДПО Н...",1958-06-20,"Деятельность прочих общественных организаций, ...",Общественные организации,"Деятельность прочих общественных организаций, ...",Общественные организации,2016-09-28 12:05:06
3,1037739471220,orthonet.ru,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,Москва,Общественная организация,СОЮЗ ПИСАТЕЛЕЙ РОССИИ,1958-12-07,Деятельность профессиональных членских организ...,Общественные организации,Деятельность профессиональных членских организ...,Профсоюзы,2005-11-24 21:00:00
4,1085200004450,vdpo-sarov.ru,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.166667,Нижегородская область,Общественная организация,"САРОВСКОЕ ""ВДПО""",1962-03-30,"Деятельность прочих общественных организаций, ...",Общественные организации,"Деятельность прочих общественных организаций, ...",Общественные организации,2021-04-15 8:10:11


In [8]:
len(concat_ngos)

9545

The aim is to add a column in concat_ngos dataset with links to web archive snapshots.

Steps:
1. For all websites in our dataset check if they are available in web archive with the help of Wayback Availability JSON API
2. Not archived -> make a snaphot, archive it now. Archived -> is the last snapshot within the last 6 months? -> If yes just get the last link, if no create new snapshot.

### Archive how to

`waybackpy` is for creating snapshots:

In [9]:
!pip install waybackpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
""" Uncomment this if you want to test how the api works
# After this code you will see the mark on the website that today was created a
# new snapshot for the unknown collection here https://web.archive.org/

from waybackpy import WaybackMachineSaveAPI

url = "ngolikeyou.ru"
user_agent = "Mozilla/5.0 (Windows NT 5.1; rv:40.0) Gecko/20100101 Firefox/40.0"

save_api = WaybackMachineSaveAPI(url, user_agent)
save_api.save()
save_api.cached_save
"""

'https://web.archive.org/web/20220621053655/https://ngolikeyou.ru/'

Since web archive creates snapshots for a web page, not a web site, we need to get a sitemap and parse all pages for each website.

There is a lib for that:

In [10]:
!pip install ultimate-sitemap-parser

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Checking for snapshots is faster with command line GET requests than with the waybackpy library (about 10 times faster):

In [11]:
# test example
availability = !curl -X GET "https://archive.org/wayback/available?url=http://vdpo-sarov.ru/"
json.loads(availability[0])['archived_snapshots']['closest']['timestamp']

'20220623103945'

One might think that making snapshots will also be faster this way, but unfortunately this method is disabled. The respecting server is down and gives 502 error

Test:

In [12]:
!curl -X POST -H "Content-Type: application/json" -d '{"url": "http://vdpo-sarov.ru/tovari", "annotation": {"id": "ngos-archive", "message": ""}}' https://pragma.archivelab.org

<html>
<head><title>502 Bad Gateway</title></head>
<body bgcolor="white">
<center><h1>502 Bad Gateway</h1></center>
<hr><center>nginx/1.4.6 (Ubuntu)</center>
</body>
</html>


Another method is "Save Page Now" button through waybackpy lib. This button however allows no more than 15 requests per minute, otherwise it blocks the user for 5 minutes (the rule was implemented by wayback web archive in 2019)

In [ ]:
# test
def save_url(url):
  save_api = WaybackMachineSaveAPI(url, user_agent)
  snapshot = save_api.save()
  return snapshot

In [ ]:
# test
from tqdm import tqdm
from random import randint
from time import sleep

for url in tqdm(urls):
  sleep(randint(10,15))
  print(save_url(url))

  2%|▏         | 1/43 [00:21<14:58, 21.39s/it]

https://web.archive.org/web/20220621053655/https://ngolikeyou.ru/


  5%|▍         | 2/43 [00:42<14:23, 21.06s/it]

https://web.archive.org/web/20220621055932/https://ngolikeyou.ru/trening


  7%|▋         | 3/43 [01:02<13:57, 20.93s/it]

...

https://web.archive.org/web/20220621071957/https://ngolikeyou.ru/tpost/30dfzgjdu1-otkrit-nabor-na-trening-dlya-veduschih-g


 98%|█████████▊| 42/43 [27:43<00:31, 31.98s/it]

https://web.archive.org/web/20220621072034/https://ngolikeyou.ru/tpost/lb4eta0cl1-zavershilsya-ocherednoi-trening-dlya-ved


100%|██████████| 43/43 [28:11<00:00, 39.33s/it]

https://web.archive.org/web/20220621072103/https://ngolikeyou.ru/tpost/fcsuhbjug1-zakonchilsya-pervii-kurs-po-art-terapii


## Archive all

In [63]:
from waybackpy import WaybackMachineSaveAPI
from tqdm import tqdm
from random import randint
from time import sleep
from usp.tree import sitemap_tree_for_homepage
from usp.exceptions import SitemapException
from datetime import date
from dateutil import relativedelta
import re



def get_urls_from_sitemap(original_url: str) -> list:
  tree = sitemap_tree_for_homepage(original_url)
  urls = [page.url for page in tree.all_pages() if page.url.startswith(original_url)]
  return urls


def how_many_months_ago(snap_timestamp: str) -> int:
  # `snap_timestamp` looks kind of like this '20220323172342'
  current = date.today()
  snap_date = date(int(snap_timestamp[:4]),  # year
                   int(snap_timestamp[4:6]),  # month
                   int(snap_timestamp[6:8]))  # day

  r = relativedelta.relativedelta(current, snap_date)
  months_difference = (r.years * 12) + r.months
  return abs(months_difference)


def find_existing_snapshot(original_url: str) -> str:
  # check if the url is available on wayback machine in web archive
  availability = !curl -X GET {"https://archive.org/wayback/available?url=" + original_url}

  try:
    response = json.loads(availability[0])
  except JSONDecodeError:
    print(availability)
    return None

  if response["archived_snapshots"] != {}:
    snapshot = response["archived_snapshots"]["closest"]
    snap_timestamp = snapshot["timestamp"]
    # the last snapshot is made in the last half a year
    if how_many_months_ago(snap_timestamp) <= 6:
      return snapshot["url"]
    else:  # if the last snapshot is older than half a year it is not good enough for us
      return None
  else:  # case when there are no snapshots at all
    return None


def save_page_now(url: str) -> str:
  user_agent = "Mozilla/5.0 (Windows NT 5.1; rv:40.0) Gecko/20100101 Firefox/40.0"
  save_api = WaybackMachineSaveAPI(url, user_agent)
  snapshot = save_api.save()
  sleep(4.5)
  return snapshot


def create_snaps(urls: list) -> list:
  snaps = []
  for url in tqdm(urls):
    web_archive_snap = find_existing_snapshot(url)
    if not web_archive_snap:
      new_snap = save_page_now(url)
      snaps.append(new_snap)
    else:
      snaps.append(web_archive_snap)
  return snaps


def has_cyrillic(text):
  return bool(re.search('[а-яА-Я]', text))


def get_all_urls(entry, original_url: str) -> list:

  if entry.bin_ssl == 1.0:
    orig_url = "https://" + original_url + "/"
    urls = get_urls_from_sitemap(orig_url)
  else:
    orig_url = "http://" + original_url + "/"
    urls = get_urls_from_sitemap(orig_url)
  if not urls:  # if there is no sitemap
    urls = [orig_url]
  
  return urls

Now we can create a column where we save the snapshot urls. But we need to do it in bulks, because we need save the data to csv file from time to time.

In [14]:
concat_ngos["webarchive_snapshots"] = "" * len(concat_ngos)

In [ ]:
# test
# А может такое быть что ssl нет, а сайтмап есть?
concat_ngos[(concat_ngos["bin_ssl"] != 1.0) & (concat_ngos["bin_sitemap"] == 1.0)]

,ogrn,website,bin_robots,bin_sitemap,bin_fb,bin_vk,bin_ig,bin_ok,bin_youtube,bin_tiktok,...,regionName,minjustForm,shortName,regDate,mainOkved,opfCombined,okvedCombined,vedSimple,websiteCreationDate,webarchive_snapshots
0,1026800000666,profobr68.ru,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Тамбовская область,Профессиональный союз,ТАМБОВСКАЯ ОБЛАСТНАЯ ОРГАНИЗАЦИЯ ОБЩЕРОССИЙСКО...,1938-11-10,Деятельность профессиональных союзов,Профсоюзные организации,Деятельность профессиональных союзов,Профсоюзы,2016-01-15 19:31:34,
2,1025200008140,vdpor52.ru,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Нижегородская область,Общественная организация,"НИЖЕГОРОДСКОЕ ОБЛАСТНОЕ ОТДЕЛЕНИЕ ВДПО, ВДПО Н...",1958-06-20,"Деятельность прочих общественных организаций, ...",Общественные организации,"Деятельность прочих общественных организаций, ...",Общественные организации,2016-09-28 12:05:06,
4,1085200004450,vdpo-sarov.ru,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,Нижегородская область,Общественная организация,"САРОВСКОЕ ""ВДПО""",1962-03-30,"Деятельность прочих общественных организаций, ...",Общественные организации,"Деятельность прочих общественных организаций, ...",Общественные организации,2021-04-15 8:10:11,
10,1027700270861,www.ujmos.ru,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Москва,Общественная организация,"РОО ""СЖМ"", СЖМ",1977-04-23,"Деятельность прочих общественных организаций, ...",Общественные организации,"Деятельность прочих общественных организаций, ...",Общественные организации,2006-09-13 20:00:00,
13,1033918500285,chessfed39.ru,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Калининградская область,Общественная организация,"КООО ""ШАХМАТНАЯ ФЕДЕРАЦИЯ""",1984-02-16,"Деятельность прочих общественных организаций, ...",Общественные организации,"Деятельность прочих общественных организаций, ...",Общественные организации,2019-12-05 13:43:18,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9500,1218600002840,gran-i.ru,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Ханты-Мансийский автономный округ - Югра,Автономная некоммерческая организация,АНО «ГРАНИ»,2021-03-16,Деятельность библиотек и архивов,Автономные некоммерческие организации,Деятельность библиотек и архивов,Издательство,2021-07-01 7:25:53,
9505,1211800004998,www.ushinsky-iro.ru,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Удмуртская республика,Автономная некоммерческая организация,ИНСТИТУТ РАЗВИТИЯ ОБРАЗОВАНИЯ ИМ. К. Д. УШИНСКОГО,2021-03-18,Образование профессиональное дополнительное,Автономные некоммерческие организации,Образование профессиональное дополнительное,Образование,2021-03-07 10:18:03,
9506,1217800042460,innovationcentre.ru,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Санкт-Петербург,Автономная некоммерческая организация,"АНО ЦЕНТР РАЗВИТИЯ ""ИННОВАЦИЯ""",2021-03-19,Предоставление прочих социальных услуг без обе...,Автономные некоммерческие организации,Предоставление прочих социальных услуг без обе...,Социальная работа,2021-05-23 18:25:03,
9527,1216600028897,прознания.рф,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,Свердловская область,Автономная некоммерческая организация,"АНО ДПО ""ПРОЗНАНИЯ""",2021-05-13,Образование профессиональное дополнительное,Автономные некоммерческие организации,Образование профессиональное дополнительное,Образование,2021-09-20 8:21:31,


In [64]:
i = 0
for entry in concat_ngos.iloc[:50].itertuples():
  original_url = entry.website
  if has_cyrillic(original_url):
    original_url = original_url.encode('idna').decode('utf-8')

  urls = get_all_urls(entry, original_url)
  snaps = create_snaps(urls)

  # save web archive links to the dataset
  concat_ngos.at[i, "webarchive_snapshots"] = snaps
  
  i += 1


2022-06-23 21:21:04,774 INFO usp.fetch_parse [61/MainThread]: Fetching level 0 sitemap from http://profobr68.ru/robots.txt...
2022-06-23 21:21:04,777 INFO usp.helpers [61/MainThread]: Fetching URL http://profobr68.ru/robots.txt...
2022-06-23 21:21:05,262 INFO usp.fetch_parse [61/MainThread]: Parsing sitemap from URL http://profobr68.ru/robots.txt...
2022-06-23 21:21:05,266 INFO usp.fetch_parse [61/MainThread]: Fetching level 0 sitemap from http://profobr68.ru/sitemap.xml...
2022-06-23 21:21:05,269 INFO usp.helpers [61/MainThread]: Fetching URL http://profobr68.ru/sitemap.xml...
2022-06-23 21:21:05,911 INFO usp.fetch_parse [61/MainThread]: Parsing sitemap from URL http://profobr68.ru/sitemap.xml...
2022-06-23 21:21:05,914 INFO usp.fetch_parse [61/MainThread]: Fetching level 1 sitemap from http://profobr68.ru/post-sitemap.xml...
2022-06-23 21:21:05,918 INFO usp.helpers [61/MainThread]: Fetching URL http://profobr68.ru/post-sitemap.xml...
2022-06-23 21:21:08,046 INFO usp.fetch_parse [61/M

TooManyRequestsError: ignored

Notes:

- 'https://prgura.ru/' throws TooManyRequests error algthough it is literally just one page, it's index is 60. It seems that domain name has changed, and makes a redirect or smth like that (???), maybe this is the reason

In [51]:
i

0

In [ ]:
a = "string"
a[2:]

'ring'

In [45]:
concat_ngos.iloc[13].webarchive_snapshots

['http://web.archive.org/web/20220324190836/http://chessfed39.ru/',
 'http://web.archive.org/web/20220324190836/http://chessfed39.ru/',
 'http://web.archive.org/web/20220324190836/http://chessfed39.ru/',
 'http://web.archive.org/web/20220324190836/http://chessfed39.ru/',
 'http://web.archive.org/web/20220324190836/http://chessfed39.ru/',
 'http://web.archive.org/web/20220324190836/http://chessfed39.ru/',
 'http://web.archive.org/web/20220324190836/http://chessfed39.ru/',
 'http://web.archive.org/web/20220324190836/http://chessfed39.ru/',
 'http://web.archive.org/web/20220324190836/http://chessfed39.ru/',
 'http://web.archive.org/web/20220324190836/http://chessfed39.ru/',
 'http://web.archive.org/web/20220324190836/http://chessfed39.ru/',
 'http://web.archive.org/web/20220324190836/http://chessfed39.ru/',
 'http://web.archive.org/web/20220324190836/http://chessfed39.ru/',
 'http://web.archive.org/web/20220324190836/http://chessfed39.ru/',
 'http://web.archive.org/web/20220324190836/http

### Save the data

In [ ]:
concat_ngos.iloc[:50].to_csv("ngos_archived_99.csv", encoding="utf-8")

ngolikeyou.ru потыкала руками. В вебархиве появляется меньше чем через полгода после создания. В 2022 фигурирует уже активнее, какие-то автоматические роботы вебархива его находят и автоматические роботы какого-то russian-web

In [ ]:
selected_ngos_data[selected_ngos_data['website'] == 'ngolikeyou.ru']

,ogrn,website,regionName,minjustForm,shortName,regDate,mainOkved,opfCombined,okvedCombined,vedSimple,websiteCreationDate
9540,1217700385539,ngolikeyou.ru,Москва,NaN,АНО ЦЕНТР СИСТЕМНОЙ ПОДДЕРЖКИ ДЛЯ ЛЮДЕЙ С ПСИХ...,2021-08-18,Предоставление прочих социальных услуг без обе...,Автономные некоммерческие организации,Автономные некоммерческие организации,АНО,2021-08-17 15:54:57


In [ ]:
!ia configure

Enter your Archive.org credentials below to configure 'ia'.

Email address: anna.klezovich24@gmail.com
Password: 

Config saved to: /root/.config/internetarchive/ia.ini


In [ ]:
import internetarchive as ia

ia.download('ngolikeyou.ru')


AttributeError: ignored

In [ ]:
item = ia.get_item('lettertowilliaml00doug')
item.download()

AttributeError: ignored

In [ ]:
!sudo pip install pymarc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 230 kB 4.2 MB/s 
  Created wheel for pymarc: filename=pymarc-4.2.0-py3-none-any.whl size=155332 sha256=3923c947e5ec024bfe261453b3b2b0359af9359337d282581cf02197acbdb9bc
  Stored in directory: /root/.cache/pip/wheels/35/c1/bc/7cbc19ab89d8fea276e17106de1231e299f0bbe73135610515
Successfully built pymarc


In [ ]:
from waybackpy import WaybackMachineSaveAPI

url = "https://github.com"
user_agent = "Mozilla/5.0 (Windows NT 5.1; rv:40.0) Gecko/20100101 Firefox/40.0"

save_api = WaybackMachineSaveAPI(url, user_agent)
save_api.save()

'https://web.archive.org/web/20220618064446/https://github.com/'

In [ ]:
save_api.cached_save

True

In [ ]:
save_api.timestamp()

datetime.datetime(2022, 6, 18, 6, 44, 46)

In [ ]:
import requests
import json
import re
from fake_useragent import UserAgent
from datetime import datetime

In [ ]:
url = "ngolikeyou.ru"
ua = UserAgent()
headers = {"user-agent": ua.chrome}

timestamp = datetime.now().timestamp()
#wburl = "https://archive.org/wayback/available?url=" + url + "&timestamp=" + str(timestamp)

In [ ]:
data

{'archived_snapshots': {},
 'timestamp': '1655541196.058648',
 'url': 'ngolikeyou.ru'}

In [ ]:
geturl = json.loads(data)

try:
    wayback = geturl['archived_snapshots']['closest']['url']
except:
    wayback = "n/a"
    print("No snapshot URL returned")

TypeError: ignored